In [ ]:
import os
import boto3
import time
import numpy as np
import sagemaker

sess = boto3.Session()
sm = sess.client("sagemaker")
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket_name = sagemaker_session.default_bucket()
jobs_folder = "jobs"
dataset_folder = "datasets"

In [ ]:
job_name = f'pytorch-smddp-dist-{time.strftime("%Y-%m-%d-%H-%M-%S-%j", time.gmtime())}'
output_path = f"s3://{bucket_name}/{jobs_folder}"

hyperparameters = {
    "epochs": 15,
    "lr": 0.01,
    "momentum": 0.9,
    "batch-size": 256,
    "model-type": "resnet18",
    "backend": "smddp",
}

In [ ]:
distribution = {"smdistributed": {"dataparallel": {"enabled": True}}}

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="training_script.py",
    source_dir="code",
    output_path=output_path + "/",
    code_location=output_path,
    role=role,
    instance_count=1,
    instance_type="ml.p4d.24xlarge",  # 'ml.p3.16xlarge', 'ml.p3dn.24xlarge', 'ml.p4d.24xlarge',
    py_version="py38",
    distribution=distribution,
    hyperparameters=hyperparameters,
)

In [ ]:
datasets = "" # s3 location of dataset

In [ ]:
estimator.fit({"train": datasets}, job_name=job_name, wait=True)